In [ ]:
import os
import cv2
import cvzone
from time import time
from cvzone.FaceDetectionModule import FaceDetector

# ========== Configuration ==========
CLASS_ID = 0  # 0 = fake, 1 = real
OUTPUT_FOLDER = 'Dataset/DataCollect'
CONFIDENCE_THRESHOLD = 0.8
SAVE_IMAGES = True
BLUR_THRESHOLD = 20
DEBUG_MODE = False
OFFSET_PERCENT_W = 10
OFFSET_PERCENT_H = 20
CAM_WIDTH, CAM_HEIGHT = 640, 480
FLOAT_PRECISION = 6

# ========== Ensure Output Folder Exists ==========
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# ========== Webcam Setup ==========
cap = cv2.VideoCapture(1)
if not cap.isOpened():
    cap = cv2.VideoCapture(0)
cap.set(3, CAM_WIDTH)
cap.set(4, CAM_HEIGHT)

# ========== Face Detector Initialization ==========
detector = FaceDetector(minDetectionCon=0.5, modelSelection=0)

# ========== Main Loop ==========
while True:
    success, frame = cap.read()
    if not success:
        print("❌ Camera not working")
        break

    img_output = frame.copy()
    frame, face_bboxes = detector.findFaces(frame, draw=False)

    blur_results = []
    annotations = []

    if face_bboxes:
        for face in face_bboxes:
            x, y, w, h = face["bbox"]
            detection_score = float(face["score"][0])

            if detection_score > CONFIDENCE_THRESHOLD:
                # Apply offsets
                offset_w = (OFFSET_PERCENT_W / 100) * w
                offset_h = (OFFSET_PERCENT_H / 100) * h

                x = max(int(x - offset_w), 0)
                y = max(int(y - offset_h * 3), 0)
                w = max(int(w + offset_w * 2), 0)
                h = max(int(h + offset_h * 3.5), 0)

                face_img = frame[y:y + h, x:x + w]

                if face_img.size > 0:
                    cv2.imshow("Face", face_img)

                    # Blur detection using Laplacian
                    blur_score = int(cv2.Laplacian(face_img, cv2.CV_64F).var())
                    blur_results.append(blur_score > BLUR_THRESHOLD)

                    # Normalize coordinates for YOLO format
                    ih, iw, _ = frame.shape
                    xc, yc = x + w / 2, y + h / 2
                    xcn = round(xc / iw, FLOAT_PRECISION)
                    ycn = round(yc / ih, FLOAT_PRECISION)
                    wn = round(w / iw, FLOAT_PRECISION)
                    hn = round(h / ih, FLOAT_PRECISION)

                    # Ensure normalized values do not exceed 1
                    xcn = min(xcn, 1)
                    ycn = min(ycn, 1)
                    wn = min(wn, 1)
                    hn = min(hn, 1)

                    # Store annotation info
                    annotations.append(f"{CLASS_ID} {xcn} {ycn} {wn} {hn}\n")

                    # Draw bounding box and info on output image
                    cv2.rectangle(img_output, (x, y, w, h), (255, 0, 0), 3)
                    cvzone.putTextRect(
                        img_output,
                        f'Score: {int(detection_score * 100)}% Blur: {blur_score}',
                        (x, y - 10),
                        scale=2,
                        thickness=2
                    )

                    if DEBUG_MODE:
                        print(f"Blur Score: {blur_score} - {'Sharp' if blur_score > BLUR_THRESHOLD else 'Blurry'}")

        # Save image and label if all detected faces are sharp
        if SAVE_IMAGES and all(blur_results) and blur_results:
            timestamp = str(time()).replace('.', '')
            img_path = f"{OUTPUT_FOLDER}/{timestamp}.jpg"
            txt_path = f"{OUTPUT_FOLDER}/{timestamp}.txt"

            cv2.imwrite(img_path, frame)
            with open(txt_path, 'a') as f:
                f.writelines(annotations)

    # Show output
    cv2.imshow("Image", img_output)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# ========== Cleanup ==========
cap.release()
cv2.destroyAllWindows()
